<a href="https://colab.research.google.com/github/canast07/EPL448-Ticket-price/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Approach 1

In [69]:
import pandas as pd
import numpy as np

Loading data in variable trainData

In [84]:
trainData = pd.read_excel("Data_Train.xlsx")

Removing problematic rows (rows that contain null fields) 

In [85]:
print(trainData.count())
print("Records with nulls: ",trainData.isnull().sum().sum())
trainData = trainData.dropna()
print("Records with nulls after cleanup: ",trainData.isnull().sum().sum())
print(trainData.count())

Airline            10683
Date_of_Journey    10683
Source             10683
Destination        10683
Route              10682
Dep_Time           10683
Arrival_Time       10683
Duration           10683
Total_Stops        10682
Additional_Info    10683
Price              10683
dtype: int64
Records with nulls:  2
Records with nulls after cleanup:  0
Airline            10682
Date_of_Journey    10682
Source             10682
Destination        10682
Route              10682
Dep_Time           10682
Arrival_Time       10682
Duration           10682
Total_Stops        10682
Additional_Info    10682
Price              10682
dtype: int64


Removing columns we consider not worth exploring

In [86]:
trainData.pop("Route") # Route is duplicate information because of source-destination
trainData.pop("Source") 
trainData.pop("Destination") # Source - Destination can be replaced with Duration time 

0        New Delhi
1         Banglore
2           Cochin
3         Banglore
4        New Delhi
           ...    
10678     Banglore
10679     Banglore
10680        Delhi
10681    New Delhi
10682       Cochin
Name: Destination, Length: 10682, dtype: object

```
1	No info
2	1 Long layover
3	2 Long layover
4	1 Short layover
5	Business class
6	Change airports
7	In-flight meal not included
8	Red-eye flight
9	No check-in baggage included
```
For column Additional_Info we have those 9 categories. We are going to replace the additional with 9 new columns (1 each category)*italicised text*

In [87]:
trainData = pd.get_dummies(trainData, columns= ['Additional_Info'],drop_first=True)
trainData.head()

,Airline,Date_of_Journey,Dep_Time,Arrival_Time,Duration,Total_Stops,Price,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,IndiGo,24/03/2019,22:20,01:10 22 Mar,2h 50m,non-stop,3897,0,0,0,0,0,0,0,1,0
1,Air India,1/05/2019,05:50,13:15,7h 25m,2 stops,7662,0,0,0,0,0,0,0,1,0
2,Jet Airways,9/06/2019,09:25,04:25 10 Jun,19h,2 stops,13882,0,0,0,0,0,0,0,1,0
3,IndiGo,12/05/2019,18:05,23:30,5h 25m,1 stop,6218,0,0,0,0,0,0,0,1,0
4,IndiGo,01/03/2019,16:50,21:35,4h 45m,1 stop,13302,0,0,0,0,0,0,0,1,0


In [96]:
trainData['Total_Stops'] = trainData['Total_Stops'].replace(['non-stop'],'0')
trainData['Total_Stops'] = trainData['Total_Stops'].replace(['1 stop'],'1')
trainData['Total_Stops'] = trainData['Total_Stops'].replace(['2 stops'],'2')
trainData['Total_Stops'] = trainData['Total_Stops'].replace(['3 stops'],'3')
trainData['Total_Stops'] = trainData['Total_Stops'].replace(['4 stops'],'4')
trainData.head()

,Airline,Date_of_Journey,Dep_Time,Arrival_Time,Duration,Total_Stops,Price,Additional_Info_1 Short layover,Additional_Info_2 Long layover,Additional_Info_Business class,Additional_Info_Change airports,Additional_Info_In-flight meal not included,Additional_Info_No Info,Additional_Info_No check-in baggage included,Additional_Info_No info,Additional_Info_Red-eye flight
0,IndiGo,24/03/2019,22:20,01:10 22 Mar,2h 50m,0,3897,0,0,0,0,0,0,0,1,0
1,Air India,1/05/2019,05:50,13:15,7h 25m,2,7662,0,0,0,0,0,0,0,1,0
2,Jet Airways,9/06/2019,09:25,04:25 10 Jun,19h,2,13882,0,0,0,0,0,0,0,1,0
3,IndiGo,12/05/2019,18:05,23:30,5h 25m,1,6218,0,0,0,0,0,0,0,1,0
4,IndiGo,01/03/2019,16:50,21:35,4h 45m,1,13302,0,0,0,0,0,0,0,1,0
